In [256]:
# import sys
# sys.path.append('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/code/cpet_articles/gathering/full-text_download_code/helper_funcs')
# from crossref_pdf_download import crossref_pdf_download
import pandas as pd
from tqdm import tqdm
import json
import requests
from pathlib import Path
import re
import random
import numpy as np
import time

In [257]:
def get_publisher_download_url(crossref_resp):
    # search for 'text-mining' in the intended-application field
    text_mining = []
    for link in crossref_response.json()['link']:
        if re.search(r'text-mining',link['intended-application']):
            text_mining.append(re.search('text-mining', link['intended-application']))
        else:
            text_mining.append(None)
            
    # find the index of match obects in crossref link information
    publisher_api_idxs = np.where(np.array(text_mining) != None)[0].tolist()
    # wiley_api_idxs
    links = [crossref_response.json()['link'][i] for i in publisher_api_idxs]
    # for now use the first link if there are more than one link
    full_text_link = links[0]['URL']
    return full_text_link

In [258]:
with open('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/code/cpet_articles/gathering/full-text_download_code/wiley/wiley_config.json') as config_file:
    wiley_token = json.load(config_file)['api_key']

In [259]:
# wiley_non_oa_status_codes = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/wiley_non_oa_status_codes.csv')
# non200 = wiley_non_oa_status_codes[wiley_non_oa_status_codes['publisher_status_code'] != 200]
# non200.shape

In [260]:
pdf_file_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs').rglob('*/*.pdf'))
pdfs = [path.stem for path in pdf_file_paths]
len(pdfs)

12659

In [261]:
articles = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/unpaywall_info.csv')
wiley_ca_articles = articles[(articles['publisher'] == 'Wiley') & (articles['is_oa'] == False)].drop_duplicates().reset_index(drop=True)

In [262]:
re_doi_suffix = re.compile(r'(?<=\d/).*')
wiley_ca_articles['doi_suffix'] = wiley_ca_articles['doi'].apply(lambda x: re_doi_suffix.search(x).group())

In [263]:
full_texts_to_download = [x for x in wiley_ca_articles['doi_suffix'].tolist() if x not in pdfs]
merge = pd.merge(pd.DataFrame({'doi_suffix': full_texts_to_download}), wiley_ca_articles, how='inner', on='doi_suffix')
merge.shape

(836, 50)

In [264]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'
accept = 'application/vnd.citationstyles.csl+json, application/vnd.crossref.unixref+xml'
dest_folder = '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/wiley_non_oa'

In [265]:
# n = random.randint(0, merge.shape[0])
# row = merge.loc[n,:]
# doi = row['doi']
# doi

In [266]:
# doi = row['doi']
# out = {'doi': doi}
# doi_url = 'https://doi.org/' + str(doi)
# crossref_headers = {'Accept': accept, 'User-Agent': user_agent}
# try:
#     crossref_response = requests.get(
#                     url = doi_url,
#                     headers=crossref_headers,
#                     allow_redirects=True,
#                     verify=True)
#     out.update({'CR_status_code': crossref_response.status_code})

#     if crossref_response.status_code == 200:
#         full_text_url = get_publisher_download_url(crossref_response)
#         publisher_headers = {
#             'User-Agent': user_agent,
#             'Wiley-TDM-Client-Token': wiley_token,
#             # 'CR-TDM-Rate-Limit': '1000',
#             # 'CR-TDM-Rate-Limit-Remaining': '76',
#             # 'CR-TDM-Rate-Limit-Reset': '1378072800'
#         }
#         publisher_response = requests.get(full_text_url, headers = publisher_headers, allow_redirects=True)
#         out.update({'publisher_status_code': publisher_response.status_code})
#         if publisher_response.status_code == 200:
#             doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
#             filename = f'{dest_folder}/{doi_suffix}.pdf'
#             with open(filename, mode='wb') as f:
#                 f.write(publisher_response.content)
# except Exception as e:
#     print(e)
#     out.update({'error': e})

# print(publisher_response.status_code)

In [267]:
log = []

In [ ]:
for idx, row in tqdm(merge.iterrows(), total=merge.shape[0]):
    doi = row['doi']
    out = {'doi': doi}
    doi_url = 'https://doi.org/' + str(doi)
    crossref_headers = {'Accept': accept, 'User-Agent': user_agent}
    try:
        crossref_response = requests.get(
                        url = doi_url,
                        headers=crossref_headers,
                        allow_redirects=True,
                        verify=True)
        out.update({'CR_status_code': crossref_response.status_code})

        if crossref_response.status_code == 200:
            full_text_url = get_publisher_download_url(crossref_response)
            publisher_headers = {
                'User-Agent': user_agent,
                'Wiley-TDM-Client-Token': wiley_token,
                # 'CR-TDM-Rate-Limit': '1000',
                # 'CR-TDM-Rate-Limit-Remaining': '76',
                # 'CR-TDM-Rate-Limit-Reset': '1378072800'
            }
            publisher_response = requests.get(full_text_url, headers = publisher_headers, allow_redirects=True)
            out.update({'publisher_status_code': publisher_response.status_code})
            if publisher_response.status_code == 200:
                doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
                filename = f'{dest_folder}/{doi_suffix}.pdf'
                with open(filename, mode='wb') as f:
                    f.write(publisher_response.content)
    except Exception as e:
        print(e)
        out.update({'error': e})
    time.sleep(0.35)

    log.append(out)

In [269]:
log_df = pd.DataFrame(log)
log_df

,doi,CR_status_code,publisher_status_code,error
0,10.1113/ep090105,200,503.0,NaN
1,10.1111/acer.14742,200,503.0,NaN
2,10.1111/apha.13785,200,503.0,NaN
3,10.1111/apha.13776,200,503.0,NaN
4,10.1111/psyp.13980,200,503.0,NaN
...,...,...,...,...
831,10.1007/s11745-005-1381-5,200,NaN,list index out of range
832,10.1136/vr.135.21.499,200,NaN,list index out of range
833,10.1113/jp282371,200,503.0,NaN
834,10.1113/ep090422,200,503.0,NaN


In [280]:
non200 = log_df[log_df['publisher_status_code'] != 200].reset_index(drop=True)
# non200.shape

In [281]:
wiley_errors = pd.merge(non200, merge, how='inner', on='doi')
wiley_errors.to_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/wiley_non_oa_status_codes.csv',
                    index=False)

In [283]:
wiley_errors.shape

(693, 53)